In [2]:
from Bio import Entrez

In [3]:
import pandas as pd
import numpy as np


In [30]:
def search(query):
    Entrez.email = 'email@example.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='1000',
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

In [31]:
studies = search('Intelligence')

In [32]:
studiesIdList = studies['IdList']

In [33]:
studiesIdList

['22577301', '15866152', '28728020', '23921675', '35892459', '35040860', '36946092', '16142448', '36905061', '7719351', '8467370', '12699709', '34600338', '26598734', '2858875', '36127407', '35964835', '31313972', '25131282', '7939558', '26868302', '28489432', '15853293', '11276900', '28065213', '36227327', '30550043', '29083200', '19780990', '30392827', '20145623', '36991773', '32428260', '34490914', '8674996', '16563422', '15347533', '1672021', '37726741', '34301553', '33146128', '26554042', '36104206', '36465956', '27464851', '33114316', '28489422', '37695935', '36203731', '36118946', '27217560', '33683205', '31610740', '27590827', '36298353', '35469208', '29199093', '36867293', '30211573', '21357223', '7739746', '35336335', '11276912', '35881682', '23410008', '33998304', '28328750', '34751066', '17295953', '28826509', '35637487', '33424960', '31748923', '35211981', '33984193', '3675282', '35035472', '36207708', '37394111', '33979390', '27064713', '24943404', '15875453', '36366091',

In [34]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'email@example.com'
    handle = Entrez.efetch(db='pubmed',
    retmode='xml',
    id=ids)
    results = Entrez.read(handle)
    return results

In [35]:
title_list= []
abstract_list=[]
journal_list = []
language_list =[]
pubdate_year_list = []
pubdate_month_list = []
studies = fetch_details(studiesIdList)
chunk_size = 500 #reduce chunksize to not exceed request limits
for chunk_i in range(0, len(studiesIdList), chunk_size):
    chunk = studiesIdList[chunk_i:chunk_i + chunk_size]
    papers = fetch_details(chunk)
    for i, paper in enumerate (papers['PubmedArticle']):
        title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])
        try:
            abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
        except:
            abstract_list.append('No Abstract')
            journal_list.append(paper['MedlineCitation']['Article']['Journal']['Title'])
            language_list.append(paper['MedlineCitation']['Article']['Language'][0])
        try:
            pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
        except:
            pubdate_year_list.append('No Data')
        try:
            pubdate_month_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month'])
        except:
            pubdate_month_list.append('No Data')
            df = pd.DataFrame(list(zip(
            title_list, abstract_list, journal_list, language_list, pubdate_year_list, pubdate_month_list
            )),
            columns=[
            'Title', 'Abstract', 'Journal', 'Language', 'Year','Month'
            ])

In [21]:
df = pd.DataFrame(list(zip(title_list,
                  abstract_list,
                  journal_list,
                  language_list,
                  pubdate_year_list,
                  pubdate_month_list)))

In [25]:
list(zip(title_list,abstract_list,journal_list,language_list,pubdate_year_list,pubdate_month_list))

[]

In [27]:
for key in [title_list,abstract_list,journal_list,language_list,pubdate_year_list,pubdate_month_list]:
    print(key)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 for key in list(title_list,abstract_list,journal_list,language_list,pubdate_year_list,pu     │
│   2 │   print(key)                                                                               │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: list expected at most 1 argument, got 6

In [36]:
[title_list,abstract_list,journal_list,language_list,pubdate_year_list,pubdate_month_list]


[
    [
        'Intelligence.',
        'Evolution of the brain and intelligence.',
        'Neuroscience-Inspired Artificial Intelligence.',
        'The relation between intelligence and religiosity: a meta-analysis and some proposed explanations.',
        'Artificial Intelligence (AI) and Internet of Medical Things (IoMT) Assisted Biomedical Systems for Intelligent Healthcare.',
        'Towards micromachine intelligence: potential of polymers.',
        'Beyond collective intelligence: Collective adaptation.',
        'Plant intelligence.',
        'Natural Intelligence as the Brain of Intelligent Systems.',
        'Questions of intelligence.',
        'On what intelligence is.',
        'Human intelligence: the brain, an electromagnetic system synchronised by the Schumann Resonance signal.',
        'Exploiting machine learning for bestowing intelligence to microfluidics.',
        'Neuronal factors determining high intelligence.',
        'Animal intelligence as encephalizati

[]